# Introduction

> ### object detection을 효율적인 실험환경을 제공하는 mmcv를 기반으로 한 mmdetection 패키지 활용

- mmcv 및 mmdetection 패키지 import

In [ ]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

- Customizing Dataset

In [ ]:
# 데이터 셋 클래스 설정
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/swin/htc_cascade_mask_rcnn_swin_small_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco.py')

- Dataset 관련 config 초기화  
 - config 구성 요소 : [MMDETECTION](https://mmdetection.readthedocs.io/en/latest/tutorials/config.html)

In [ ]:
PREFIX = '../../input/data/'

# dataset 바꾸기
# json file경로와 이미지 경로 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train_all.json'
cfg.data.train.seg_prefix=PREFIX

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/version'

cfg.evaluation.metric = ['bbox']
cfg.log_config.interval = 200

cfg.checkpoint_config = dict(type='CheckpointHook', interval=1, max_keep_ckpts=5)

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)


In [ ]:
cfg.lr_config = dict(policy='CosineAnnealing',warmup='linear',warmup_iters=3000,
                    warmup_ratio=0.0001, min_lr_ratio=1e-7)

- Model builder

In [ ]:
model = build_detector(cfg.model)

- Dataset class 초기화

In [ ]:
datasets = [build_dataset(cfg.data.train)]

- Training Model

In [ ]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)